In [1]:
from datasets import load_dataset, DatasetDict

timit = DatasetDict()

timit["train"] = load_dataset("timit_asr", data_dir="./timit", split="train")
timit["test"] = load_dataset("timit_asr", data_dir="./timit", split="test")

timit

/Users/henrytsyu/imperial/segp/model/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 1680
    })
})

In [2]:
model_name = "openai/whisper-tiny.en"

In [3]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name)

In [4]:
from transformers import WhisperTokenizerFast

tokenizer = WhisperTokenizerFast(
    tokenizer_file="tokenizer/tokenizer.json",
)

In [5]:
# Ensure sampling rate is 16kHz
timit["train"][0]

{'file': '/Users/henrytsyu/imperial/segp/model/timit/data/TRAIN/DR1/FCJF0/SA1.WAV',
 'audio': {'path': '/Users/henrytsyu/imperial/segp/model/timit/data/TRAIN/DR1/FCJF0/SA1.WAV',
  'array': array([ 3.05175781e-05, -3.05175781e-05,  6.10351562e-05, ...,
         -3.05175781e-05, -1.52587891e-04, -2.44140625e-04]),
  'sampling_rate': 16000},
 'text': 'She had your dark suit in greasy wash water all year.',
 'phonetic_detail': {'start': [0,
   3050,
   4559,
   5723,
   6642,
   8772,
   9190,
   10337,
   11517,
   12500,
   12640,
   14714,
   15870,
   16334,
   18088,
   20417,
   21199,
   22560,
   22920,
   23271,
   24229,
   25566,
   27156,
   28064,
   29660,
   31719,
   33360,
   33754,
   34715,
   36080,
   36326,
   37556,
   39561,
   40313,
   42059,
   43479,
   44586],
  'stop': [3050,
   4559,
   5723,
   6642,
   8772,
   9190,
   10337,
   11517,
   12500,
   12640,
   14714,
   15870,
   16334,
   18088,
   20417,
   21199,
   22560,
   22920,
   23271,
   24229,
  

In [6]:
def prepare_dataset(example):
    # load audio data as 16kHz
    audio = example["audio"]

    # compute log-Mel input features from input audio array
    example["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    example["labels"] = tokenizer(" ".join(example["phonetic_detail"]["utterance"])).input_ids
    return example

In [7]:
timit_clean = timit.map(prepare_dataset, remove_columns=timit.column_names["train"], num_proc=1)

In [8]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model_name)
model.train()
model.freeze_encoder()
model.get_decoder().embed_tokens.num_embeddings = tokenizer.vocab_size
model.get_decoder().embed_tokens.padding_idx = tokenizer.pad_token_type_id
model.proj_out.out_features = tokenizer.vocab_size
model.config.pad_token_id = tokenizer.pad_token_type_id

# You can verify parameters that will be updated
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {trainable_params}")

model


Number of trainable parameters: 29551872


WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 384, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(384, 384, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 384)
      (layers): ModuleList(
        (0-3): 4 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=384, out_features=384, bias=False)
            (v_proj): Linear(in_features=384, out_features=384, bias=True)
            (q_proj): Linear(in_features=384, out_features=384, bias=True)
            (out_proj): Linear(in_features=384, out_features=384, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          

In [9]:
timit_clean

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 1680
    })
})

In [10]:
import torch

from dataclasses import dataclass
from typing import Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    feature_extractor: WhisperFeatureExtractor
    tokenizer: WhisperTokenizerFast
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


In [11]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    feature_extractor=feature_extractor,
    tokenizer=tokenizer,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

In [12]:
import evaluate

metric = evaluate.load("wer")

In [13]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_type_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


In [14]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir=f"./{model_name}-timit",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-4,
    warmup_steps=200,
    max_steps=2000,
    gradient_checkpointing=True,
    fp16=False,
    eval_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    load_best_model_at_end=False,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

In [15]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=timit_clean["train"],
    eval_dataset=timit_clean["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [16]:
trainer.train()

You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/Users/henrytsyu/imperial/segp/model/venv/lib/python3.12/site-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
500,0.444400,0.468259,522.735989
1000,0.266700,0.443740,521.964300


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/Users/henrytsyu/imperial/segp/model/venv/lib/python3.12/site-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 357, 366, 438, 532, 685, 705, 796, 930, 1058, 1220, 1267, 1279, 1303, 1343, 1377, 1391, 1635, 1782, 1875, 2162, 2361, 2488, 3467, 4008, 4211, 4600, 4808, 5299, 5855, 6329, 7203, 9609, 9959, 10563, 10786, 11420, 11709, 11907, 13163, 13697, 13700, 14808, 15306, 16410, 16791, 17992, 19203, 19510, 20724, 22305, 22935, 27007, 30109, 30420, 33409, 34949, 40283, 40493, 40549, 47282, 49146, 50257, 50357, 50358, 50359, 50360, 50361]}. You are seeing this w

KeyboardInterrupt: 

In [17]:
input_features = torch.tensor(timit_clean["test"][0]["input_features"]).unsqueeze(0)

model_trained = WhisperForConditionalGeneration.from_pretrained(training_args.output_dir + "/checkpoint-1000")

generated_ids = model_trained.generate(input_features=input_features)

transcription = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
transcription

'sh iy tcl ae dcl d axr dcl d aa r kcl k s ux tcl t ih n gcl r iy s iy epi epi epi ao sh epi epi ao dx er q ao dh iy axr h# h# h# h# h# h# h# h# h# h# h# sh iy epi aa r kcl k epi ax h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# dh ax tcl h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# dh ax tcl s epi epi dh ax tcl s epi epi dh ax tcl h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# dh ax tcl s epi dh ax tcl s epi epi dh ax tcl h# h# h# h# h# h# h# h# h# tcl s epi dh ax tcl s epi epi dh ax tcl s epi epi dh ax tcl s epi epi epi dh ax tcl s epi epi epi dh ax tcl h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# h# t

In [18]:
timit["test"][0]["phonetic_detail"]["utterance"]

['h#',
 'sh',
 'iy',
 'hv',
 'ae',
 'dcl',
 'd',
 'y',
 'er',
 'dcl',
 'd',
 'aa',
 'r',
 'kcl',
 'k',
 's',
 'uw',
 'dx',
 'ih',
 'ng',
 'gcl',
 'g',
 'r',
 'iy',
 's',
 'iy',
 'w',
 'aa',
 'sh',
 'epi',
 'w',
 'aa',
 'dx',
 'er',
 'q',
 'ao',
 'l',
 'y',
 'iy',
 'axr',
 'h#']

In [21]:
len(list(filter(lambda x: x != "h#", transcription.split()))), len(timit["test"][0]["phonetic_detail"]["utterance"])

(151, 41)